# Analyzing borrowers’ risk of defaulting

The project is to prepare a report for a bank’s loan division. We will need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

# Step 1. Opening File & Observing Data

In [34]:
import pandas as pd 
import numpy as np

df = pd.read_csv ('/datasets/credit_scoring_eng.csv')
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


**Conclusion: at first glance on the data using the panda's info() method, we can see that two columns have less values than the rest, more specifically the columns: 'days_employed', and 'total_income', with values 19351, compared to 21525. The ~2000 null values in each column will need to be filled. But first let's describe our takeaways for each column in our dataset.**


**Children:** no obvious issues. 

**Days employed:** in addition to null values, there are also negative values. These need to be converted to positive values. Type can also be converted from float to int, since we're just counting days. 

**DOB years:** no obvious issues. 

**Education:** strings aren't normalized. We need to normalize all string values. 

**Education ID:** Education corresponds to an ID. We can use this ID to analyze data rather than using the education column, to prevent errors from potentially malformed strings. Other than that, no obvious issues. 

**Family status/Family ID:** Family status is similar to education, a categorical variable corresponding to an ID. Both have no obvious issues. 

**Gender:** no obvious issues. 

**Income Type:** no obvious issues. 

**Debt:** binary with 0 representing no defaults on loans, and 1 representing past default(s) on loan(s). No obvious issues. 

**Total Income:** null values as mentioned previously. 

**Purpose:** random string inputted by user. This needs to be caregorized for a better understanding.


Now that we have a good idea of what the main issues are with our data we will need to take care of them before we can start our analysis. Our first step will be to fill in missing values in the columns that have null values; 'days_employed' and 'total_income'. Since these are both quantitative variables we will fill them in with the median value. The reason that will fill these values using the median and not the mean is because we can identify major ourliers and thus the median is a better indicator. Lastly, we will need to changed the negative values to positive values in the 'days_employed' column since a negative value is unrealistic and probably happened due to some technincal erros.

# Step 2. Data Preprocessing 

# Data Exploration

In [35]:
(df.isnull().sum()/len(df)).round(4) 
#getting the % of missing values and rounding accordingly 

print(df['days_employed'].isna().sum() * 100 / len(df)) 
#finding the percentage of missing valus 

print ()

print(df['total_income'].isna().sum() * 100 / len(df)) 
#confirming that both columns have the exact % 

print ()

print(len(df.loc[df['days_employed'].isna() & df['total_income'].isna()])) 
#confirming the pattern that we identified

print ()

days_employed_na = df.loc[df['days_employed'].isna()] 
#here we are creating a variable that will store our na data
days_employed_not_na = df.loc[~df['days_employed'].isna()] 
#here we are creating a variable that will store our not na data

10.099883855981417

10.099883855981417

2174



# Understanding Missing Data 

In [36]:
print(days_employed_na.groupby('family_status')['dob_years'].count() * 100 / len(days_employed_na)) 
#we can tell that most of the missing values are related to married individuals 57% 

print () 

print(days_employed_na.groupby('children')['dob_years'].count() * 100 / len(days_employed_na)) 
#we can tell that most of the missing values are related to individuals that have no kids 66%
 
print () 

print(days_employed_na.groupby('debt')['dob_years'].count() * 100 / len(days_employed_na)) 
#we can tell that most of the missing values are related to individuals that have no debt 92%

print () 

print(days_employed_na.groupby('gender')['dob_years'].count() * 100 / len(days_employed_na)) 
#we can tell that most of the missing values are related to women 68%

family_status
civil partnership    20.331187
divorced              5.151794
married              56.899724
unmarried            13.247470
widow / widower       4.369825
Name: dob_years, dtype: float64

children
-1      0.137994
 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 4      0.321987
 5      0.045998
 20     0.413983
Name: dob_years, dtype: float64

debt
0    92.180313
1     7.819687
Name: dob_years, dtype: float64

gender
F    68.26127
M    31.73873
Name: dob_years, dtype: float64


# Data Transformation

In [37]:
print(df['education'].value_counts(dropna=False)) 
#checking all unique values in the education column

df['education']=df['education'].str.lower() 
#fixing capitalization issue

print () 

print(df['education'].value_counts()) 
#confirming that changes have been made successfully

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


In [38]:
print(df['gender'].value_counts()) 
#checking all unique values in the gender column

print(df['gender'].value_counts() * 100 / len(df)) 
#finding the % of appearance for each single value

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64


Here we can identify a third gender called XNA, this is not a valid gender. Since is such only one value (0.004% of our column's data) we can delete it.

In [39]:
df= df[df['gender']!= 'XNA'] 
#deleting invalid value

print(df['gender'].value_counts()) 
#confirming that changes have been made successfully

F    14236
M     7288
Name: gender, dtype: int64


In [40]:
print(df['children'].value_counts()) 
#checking all unique values in the children column

print ()

print(df['children'].value_counts()  * 100 / len(df)) 
#finding the % of appearance for each single value

df['children'] = df['children'].replace(20, 0) 
#doing the corresponding changes/replacements

df['children'] = df['children'].replace(-1, 0) 
#doing the corresponding changes/replacements

print ()

print(df['children'].value_counts()) 
#confirming that changes have been made successfully

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 0     65.731277
 1     22.384315
 2      9.547482
 3      1.533172
 20     0.353094
-1      0.218361
 4      0.190485
 5      0.041814
Name: children, dtype: float64

0    14271
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


In [41]:
all_negative_values = (len(df.loc[df['days_employed'] < 0])) 
#finding the number of negative values to use later

all_values = df['days_employed'].size 
#finding the number of total values to use later

amount_of_neg_values= all_negative_values/all_values 
#dividing the negatives with the total to find the % of neg values

print (f'The percentage of negative values is: {amount_of_neg_values:.0%}') 
#printing the % result using f strings

The percentage of negative values is: 74%


# Data Type Replacement

In [42]:
df['days_employed'] = df['days_employed'].abs() 
#fixing negative values to positive

print(df['days_employed'].head(15)) 
#confirming that changes have been made successfully by printing first 15 rows

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
Name: days_employed, dtype: float64


In [43]:
mean_dob_years = (df['dob_years'].mean()) 
#creating a variable to store the mean of the dob_years column

df['dob_years'] = df['dob_years'].replace(0, mean_dob_years) 
#replacing 0 inputs with the mean

df['dob_years'] = df['dob_years'].astype (int) 
#since the mean was a float, we are converting the column back to integer

print (df[df['dob_years']==0])
#confirming that changes have been made successfully since it results to an empty dataframe

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


# Processing Duplicates

In [44]:
print(df.duplicated().sum()) 
#checking for duplicates in our dataset

duplicated_df = df[df.duplicated()] 
#creating a variable with all of our duplicates 

df = df.drop_duplicates() 
#confirming that changes have been made successfully since it results to empty dataframe

71


# Categorizing Data


In [45]:
def age_category (age): 
    #creating a function to organize age to 4 different categories
    
    if age <= 18:
        return 'children'

    elif age <= 30:
        return 'young adult'
    
    elif age <= 60:
        return 'middle-aged adult'
    
    else:
        return 'old adult'
    
df['age_group'] = df ['dob_years'].apply(age_category) 
#applying our function to our dob_years column

In [46]:
real_estate_stuff = ['purchase of the house', 'housing transactions', 'purchase of the house for my family', 'buy real estate', 'buy commercial real estate', 'buy residential real estate estate', 'construction of own property', 'property', 'building a property', 'transactions with commercial real estate', 'buy residential real estate', 'building a real estate', 'housing','transactions with my real estate', 'purchase of my own house', 'real estate transactions', 'buying property for renting out', 'housing renovation']
car_stuff = ['car purchase', 'buying a second-hand car', 'buying my own car', 'cars', 'second-hand car purchase', 'car', 'to own a car', 'purchase of a car', 'to buy a car']
edu_stuff = ['supplementary education', 'education', 'to become educated', 'getting an education', 'to get a supplementary education', 'getting higher education', 'profile education', 'university education', 'going to university']
wed_stuff = ['to have a wedding', 'having a wedding', 'wedding ceremony']

def purpose_grouped (row):
    #creating a function to categorize our data based on loan purpose
        
    purpose = row['purpose']
    
    if purpose in real_estate_stuff:
        return 'real estate'
    
    elif purpose in car_stuff:
        return 'car'
    
    elif purpose in edu_stuff:
        return 'education'

    elif purpose in wed_stuff:
        return 'wedding'
    
    else:
        return 'other'
    
df['purpose_group'] = df.apply(purpose_grouped, axis=1)
#applying the function to our dataset and adding column

In [47]:
def children_group (children): 
    #creating a function to categorize our data based on kids vs no kids
        
    if children > 0:
        return 'kids'

    else:
        return 'no kids'
    
df['children_group'] = df['children'].apply(children_group) 
#applying the function to our dataset and adding column

In [48]:
def categorization (income): 
    #creating a function to categorize classes based on the amount of income
        
    if income <= 50000:
        return 'lower class'
    
    elif income <= 100000:
        return 'lower middle class'
            
    elif income <= 150000:
        return 'upper middle class'
           
    elif income > 150000:
        return 'upper class'
    
df['income_group'] = df['total_income'].apply(categorization) 
print (df['income_group'].value_counts())
#applying the function to our dataset and adding column

lower class           18030
lower middle class     1221
upper middle class       71
upper class              28
Name: income_group, dtype: int64


# Filling In Missing Values

In [49]:
grouped_df = df.groupby(['family_status', 'age_group', 'education'])['total_income'].median() 
#finding the total_income median by grouping the columns; family_status, age_group and education

def filling_na (row): 
    #writing a function that will fill in missing values by the grouped median
    median_total_income = grouped_df[row['family_status']][row['age_group']][row['education']]
    return median_total_income 

df['total_income_without_na'] = df.apply(filling_na, axis=1) 
#applying to every row in our new column

df['total_income'] = df['total_income'].fillna(df['total_income_without_na']) 
#replacing the total_income column

df ['total_income'] = df['total_income'].fillna(df['total_income'].median())
#replacing the total_income column with the median

df ['total_income_without_na'] = df['total_income_without_na'].fillna(df['total_income_without_na'].median())
#replacing the total_income_without_na column with the median


grouped_df2 = df.groupby(['family_status', 'age_group', 'education'])['days_employed'].median() 
#finding the days_employed median by grouping the columns; family_status, age_group and education

def filling_na2 (row): 
    #writing a function that will fill in missing values by the grouped median
    median_total_days_employed = grouped_df2[row['family_status']][row['age_group']][row['education']]
    return median_total_days_employed 

df['total_days_employed_without_na'] = df.apply(filling_na2, axis=1) 
#applying to every row in our new column

df['days_employed'] = df['days_employed'].fillna(df['total_days_employed_without_na']) 
#replacing the days_employed column

# Conclusion

The questions that need to be answered are:

- Is there a relation between having kids and repaying a loan on time?
- Is there a relation between marital status and repaying a loan on time?
- Is there a relation between income level and repaying a loan on time?
- How do different loan purposes affect on-time repayment of the loan?

# Step 3. Answering the Questions

**Is there a relation between marital status and repaying a loan on time?**

As we can see there's clear correlation that the more kids an individual has the more chances to be in debt. We used the aggfunc parameter on the pivot_table() method and we can tell by looking at the mean that proportionally more individuals with kids have debt. More specifically 9.2% are the chances of someone being in debt while also being a parent vs 7.5% of being in debt without having a kid - almost 2% difference. We can explain this by saying that a parent has additional financial obligations thus might have more chances on taking out a loan.

In [50]:
data_pivot_children = df.pivot_table(index='children_group', values='debt', aggfunc=['count', 'sum', 'mean'])
#creating pivot table with the family status and another to compare
print(data_pivot_children)

print ()

data_pivot_children2 = df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
#calculating default-rate based on the number of children
print(data_pivot_children2)

                count   sum      mean
                 debt  debt      debt
children_group                       
kids             7240   669  0.092403
no kids         14213  1072  0.075424

          count   sum      mean
           debt  debt      debt
children                       
0         14213  1072  0.075424
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000


**Is there a correlation between family status and paying back on time?**

As we can see there's clear correlation that being married has a higher chance of being in debt. Potentially that could be because as a couple the possibility of one person being in debt increases.

In [51]:
data_pivot_family = df.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean'])
#calculating default-rate based on family status

print(data_pivot_family)

                   count  sum      mean
                    debt debt      debt
family_status                          
civil partnership   4150  388  0.093494
divorced            1195   85  0.071130
married            12339  931  0.075452
unmarried           2810  274  0.097509
widow / widower      959   63  0.065693


**Is there a correlation between income level and paying back on time?**

Here we also used the aggfunc parameter on the pivot_table() method and we can tell by looking at the mean that proportionally more individuals that fall under the lower class categories are in debt. Expected but always good to be able to justify concepts using data.

In [52]:
data_pivot_income = df.pivot_table(index='income_group', values='debt', aggfunc=['count', 'sum', 'mean'])
#calculating default-rate based on income level 

print(data_pivot_income)

                    count   sum      mean
                     debt  debt      debt
income_group                             
lower class         18030  1479  0.082030
lower middle class   1221    86  0.070434
upper class            28     2  0.071429
upper middle class     71     4  0.056338


**How does credit purpose affect the default rate?**

Lastly, also by calling the aggfunc parameter on the pivot_table() method and looking at the mean we can tell that individuals that are in debt are usually found for either car or education purposes - 9.3% and 9.2& respectively. However, the wedding category falls close to these specificially at 8% and lastly we observe the real estate category which comes at 7.2%. 

In [53]:
data_pivot_income = df.pivot_table(index='purpose_group', values='debt', aggfunc=['count', 'sum', 'mean'])
#calculating default-rate based on the purpose of the loan 

print(data_pivot_income)

               count  sum      mean
                debt debt      debt
purpose_group                      
car             4306  403  0.093590
education       4013  370  0.092200
real estate    10810  782  0.072340
wedding         2324  186  0.080034


# Step 4. General Conclusion

We identified that default rates for car loans and education loans are the highest (above 9%), followed by wedding loans (about 8%), and the lowest being real estate loans (just above 7%). Customers whose income level is categorized in either lower class/lower middle class are more likely to default on their loans vs customers whose income level is categorized in either upper middle class/upper class. Additionally, the highest default rates with respect to family status belong to the "unmarried" and "civil partnership" groups (both over 9%), followed by the married and divorced groups (7.5% and 7.1% respectively). Customers with kids are more likely (9.2%) to default on their loan than customers without kids(7.5%), especially customers with 4 kids have the highest chance to default (9.7%). 

After the above results the bank institution will find the categories with lower default rates more credit worthy, more specifically the ones that meet: no kids, high income, widowed, and taking a loan for real estate purpose. On the other hand the most risky customers are the ones from from high default rate categories such having kids, low incone, unmarried or in a civil partnership, and taking a car or education loan. 

**Project Recap**

- We worked with missing values. 


- We used the apply() method to fill in missing values. Firstly though, we created a function that organized our main interest column by 'family_status', 'age_group', 'education' and found its median accordingly. We decided to replace missing values by the median instead of the mean because it is common in salaries to have major outliers. As shown also in our case the salaries varied from 3,306 to 36,2496 (per month) so we can clearly identify that we have inviduals of all types of social classes thus we have outliers, and median is the best for this scenario. We applied similar logic to the missing values of the days_employed mcolumn. 


- Additionally, the duplicated() and drop_duplicates() methods were used to find and delete duplicates accordingly. The reason that we used these methods is because are quick and reliable. We also stored all the duplicates in a new variable for having them in case we would need to go back to these.


- We created dictionaries for family status, children, and debt. 


- We categorized the loan purposes into car, education, real estare and wedding purchase. Additionally, we categorized income, age, kids vs no kids for a cleaner and more efficient analysis. 


- Lastly, we used pivot table/s to support our analysis and come to conclusions.